In [ ]:
!pip install langchain
!pip install pypdf
!pip install sentence_transformers
!pip install llama-cpp-python
!pip install huggingface_hub
!pip install --upgrade --quiet langchain-pinecone langchain langchain-community langchain-core

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import Pinecone
from langchain.chains import RetrievalQAWithSourcesChain
import os

In [ ]:
# Import required libraries to run llm locally

from langchain.llms import LlamaCpp
from huggingface_hub import hf_hub_download

In [ ]:
#Setup the Environment
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""
os.environ["PINECONE_API_KEY"] = ""

In [ ]:
model_name_or_path = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
model_basename = "tinyllama-1.1b-chat-v1.0.Q5_K_M.gguf"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
# Loading model

llm = LlamaCpp(
    model_path=model_path,
    max_tokens=512,
    #n_gpu_layers=n_gpu_layers,
    #n_batch=n_batch,
    temperature=0.2,
    n_ctx=5000,
    streaming = True,
    verbose=False,
)

In [ ]:
# initialize pinecone

Pinecone.api_key = os.environ.get("PINECONE_API_KEY")
index_name = "nagp"

In [ ]:
# Load the document

loader = PyPDFLoader("/content/Assignment_Support_Document.pdf")
data = loader.load()
data

In [ ]:
#Split the Text into Chunks

text_splitter=RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
docs=text_splitter.split_documents(data)
print(len(docs))
docs[0]

In [ ]:
# Generate the Embeddings for data retrieved from pdf document

embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# Store embeddings into pinecone vector DB

docsearch=Pinecone.from_documents(documents= docs, embedding=embeddings, index_name = index_name)

In [ ]:
# Similarity search from pinecone DB
query = "what is Direct taxes"
resp = docsearch.similarity_search(query)
resp

In [ ]:
# Create a chain to query through LLM from pinecone

chain=RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=docsearch.as_retriever())

In [ ]:
import langchain
langchain.debug = True
chain({"question":query}, return_only_outputs=True)